### Github Notebook Repo: https://github.com/ppalmes

# [DFA Preliminaries](./DFASlide.pdf)

In [81]:
nprocs()==1 && addprocs()

@everywhere logis(x) = 1 ./ (1 .+ exp(-x))
@everywhere dlogis(x) = logis(x) .* (1 .- logis(x))

# @everywhere logis(x) = sign(x)
# @everywhere dlogis(x) = 0


@everywhere function initParams(x,y,n_hidden,max_iter)
    w1 = randn(n_hidden,size(x,1))
    w2 = randn(n_hidden,n_hidden)
    w3 = randn(size(y,1),n_hidden)  
    B1 = rand(n_hidden,1)
    B2 = rand(n_hidden,1)      
    errors = zeros(max_iter)
    outputs = zeros(size(y,2),max_iter)
    return (w1,w2,w3,errors,outputs,B1,B2)
end

@everywhere function ffward(x,y,w1,w2,w3)
    a1 = w1 * x
    h1 = logis(a1)
    a2 = w2 * h1
    h2 = logis(a2)
    ay = w3 * h2
    yhat = logis(ay)
    err = yhat - y
    return(a1,h1,a2,h2,ay,yhat,err)
end

@everywhere function updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err;lr=0.2)
    w1 .-= δa1 * x' .* lr
    w2 .-= δa2 * h1' .* lr
    w3 .-= err * h2' .* lr
end

@everywhere function bp(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,_,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter   
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (w3' * err) .* dlogis(a2)
        δa1 = (w2' * δa2) .* dlogis(a1)
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function fa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*δa2) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function dfa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*err) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end




@everywhere function ifa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa1 = (B1.*err) .* dlogis(a1) 
        δa2 = (w2 * δa1) .* dlogis(a2) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end



Any, Any, Any, Any) in module Main at In[79]:95 overwritten at In[81]:81.


In [82]:
x = [1 1 ; 0 1 ; 1 0 ; 0 0]'
y = [0 1 1 0]
n_hidden = 30; # Number of hidden units
max_iter = 1000; #Number of learning steps
trials = 30;

dfa_err=@parallel (hcat) for _=1:trials
    (_,dfa_errors) = dfa(x,y,n_hidden,max_iter)
    dfa_errors
end
(dfa_yhat,_) = dfa(x,y,n_hidden,max_iter)

bp_err = @parallel (hcat) for _=1:trials
    (_,bp_errors) = bp(x,y,n_hidden,max_iter)
    bp_errors
end
(bp_yhat,_) = bp(x,y,n_hidden,max_iter)

ifa_err=@parallel (hcat) for _=1:trials
    (_,ifa_errors) = ifa(x,y,n_hidden,max_iter)
    ifa_errors
end
(ifa_yhat,_) = ifa(x,y,n_hidden,max_iter);

fa_err=@parallel (hcat) for _=1:trials
    (_,fa_errors) = fa(x,y,n_hidden,max_iter)
    fa_errors
end
(fa_yhat,_) = fa(x,y,n_hidden,max_iter);

using Plots
l = @layout [
    a b
    c d
    e f
]
gr()
#init_notebook(true)
p1=plot(dfa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="DFA")
p2=plot(ifa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="IFA")
p3=plot(fa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="FA")
p4=plot(bp_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="BP")
p5=plot([dfa_err[:,1] ifa_err[:,1] fa_err[:,1] bp_err[:,1]],label=["DFA" "IFA" "FA" "BP"],title="First Trial Error")
p6=plot([mean(dfa_err,2) mean(ifa_err,2) mean(fa_err,2) mean(bp_err,2)],label=["DFA" "IFA" "FA" "BP"],title="Mean Trials Error")
plot(p1,p2,p3,p4,p5,p6,size=(900,900),layout=l)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 DFA 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.8; stroke-opacity:1; fill:none" points="
 39.4926,280.431 39.8995,61.0933 40.3065,208.484 40.7135,40.3265 41.1205,238.434 41.5275,35.4194 41.9344,242.042 42.3414,35.0136 42.7484,242.117 43.1554,34.791 
 43.5623,242.078 43.9693,34.5831 44.3763,242.022 44.7833,34.3856 45.1902,241.96 45.5972,34.1974 46.0042,241.895 46.4112,34.0184 46.8182,241.832 47.2251,33.8482 
 47.6321,241.773 48.0391,33.6871 48.4461,241.719 48.853,33.5352 49.26,241.671 49.667,33.3927 50.074,241.632 50.481,33.26 50.8879,241.601 51.2949,33.1374 
 51.7019,241.58 52.1089,33.0253 52.5158,241.569 52.9228,32.9239 53.3298,241.57 53.7368,32.8335 54.1438,241.583 54.5507,32.7543 54.9577,241.609 55.3647,32.6866 
 55.7717,241.649 56.1786,32.6305 56.5856,241.703 56.9926,32.586 57.3996,241.772 57.8065,32.5533 58.2135,241.857 58.6205,32.5322 59.0275,241.959 59.4345,32.5226 
 59.8414,242.078 60.2484,32.5245 60.6554,242.215 61.0624,32.5375 61.4693,242.371 61.8763,32.5613 62.2833,242.545 62.6903,32.5957 63.0973,242.738 63.5042,32.6401 
 63.9112,242.949 64.3182,32.6942 64.7252,243.178 65.1321,32.7573 65.5391,243.425 65.9461,32.829 66.3531,243.688 66.76,32.9088 67.167,243.965 67.574,32.9961 
 67.981,244.257 68.388,33.0903 68.7949,244.56 69.2019,33.1912 69.6089,244.873 70.0159,33.2982 70.4228,245.195 70.8298,33.4111 71.2368,245.524 71.6438,33.5296 
 72.0508,245.858 72.4577,33.6536 72.8647,246.197 73.2717,33.783 73.6787,246.539 74.0856,33.9178 74.4926,246.883 74.8996,34.0582 75.3066,247.228 75.7136,34.2042 
 76.1205,247.574 76.5275,34.3563 76.9345,247.921 77.3415,34.5146 77.7484,248.269 78.1554,34.6795 78.5624,248.617 78.9694,34.8517 79.3763,248.966 79.7833,35.0314 
 80.1903,249.317 80.5973,35.2193 81.0043,249.668 81.4112,35.4161 81.8182,250.021 82.2252,35.6223 82.6322,250.376 83.0391,35.8387 83.4461,250.732 83.8531,36.066 
 84.2601,251.091 84.6671,36.305 85.074,251.45 85.481,36.5565 85.888,251.811 86.295,36.8213 86.7019,252.174 87.1089,37.1004 87.5159,252.536 87.9229,37.3947 
 88.3299,252.898 88.7368,37.7051 89.1438,253.26 89.5508,38.0328 89.9578,253.62 90.3647,38.3788 90.7717,253.977 91.1787,38.7443 91.5857,254.331 91.9926,39.1306 
 92.3996,254.682 92.8066,39.5391 93.2136,255.028 93.6206,39.9711 94.0275,255.368 94.4345,40.4285 94.8415,255.703 95.2485,40.9128 95.6554,256.031 96.0624,41.426 
 96.4694,256.352 96.8764,41.9702 97.2834,256.665 97.6903,42.5476 98.0973,256.97 98.5043,43.1607 98.9113,257.266 99.3182,43.8122 99.7252,257.554 100.132,44.5052 
 100.539,257.832 100.946,45.2429 101.353,258.101 101.76,46.0289 102.167,258.359 102.574,46.8674 102.981,258.607 103.388,47.7627 103.795,258.845 104.202,48.7199 
 104.609,259.071 105.016,49.7445 105.423,259.285 105.83,50.8429 106.237,259.487 106.644,52.0222 107.051,259.675 107.458,53.2904 107.865,259.848 108.272,54.6568 
 108.679,260.006 109.086,56.132 109.493,260.145 109.9,57.7279 110.307,260.264 110.714,59.4586 111.121,260.361 111.528,61.3404 111.935,260.431 112.342,63.3919 
 112.748,260.472 113.155,65.6349 113.562,260.476 113.969,68.095 114.376,260.44 114.783,70.8018 115.19,260.354 115.597,73.79 116.004,260.21 116.411,77.1 
 116.818,259.996 117.225,80.7791 117.632,259.698 118.039,84.8825 118.446,259.3 118.853,89.4736 119.26,258.778 119.667,94.6251 120.074,258.105 120.481,100.418 
 120.888,257.248 121.295,106.938 121.702,256.165 122.109,114.269 122.516,254.804 122.923,122.481 123.33,253.107 123.737,131.601 124.144,251.013 124.551,141.578 
 124.958,248.472 125.365,152.242 125.772,245.464 126.179,163.263 126.586,242.033 126.993,174.169 127.4,238.318 127.807,184.417 128.214,234.556 128.621,193.531 
 129.028,231.04 129.435,201.216 129.842,228.049 130.249,207.403 130.656,225.771 131.062,212.207 131.469,224.263 131.876,215.852 132.283,223.47 132.69,218.593 
 133.097,223.265 133.504,

# [MNIST with BP vs DFA](./mnist.ipynb)

In [60]:
mean(fa_err,2)

500×1 Array{Float64,2}:
 1.97244 
 2.01976 
 1.96826 
 2.01415 
 1.97317 
 2.0098  
 1.97192 
 2.00395 
 1.96786 
 2.00018 
 1.96239 
 1.99694 
 1.95628 
 ⋮       
 0.464423
 0.46257 
 0.459388
 0.456857
 0.452786
 0.449498
 0.444789
 0.441679
 0.437848
 0.436335
 0.433968
 0.433438